# Imports

In [101]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn import datasets, svm, tree, metrics
from sklearn.cross_decomposition import CCA
from sklearn import preprocessing
import matplotlib.pyplot as plt
from pandas import DataFrame
import numpy as np
import itertools
import random
import json
import sys
import os.path

# Load data set into script

In [60]:
with open('../../../Downloads/codeforces-scraper-master/cfspider/CodeForceDataSet/processed_data.json') as data:
    jdata = json.load(data)

# Extract Data

In [100]:
dictionary = []
rows = []
index = []
for problem in jdata:
    for submissions in problem['Submissions']:
        for submission in submissions:
            code = submissions[submission]['Code'][0]
            dictionary.append(code)
            if problem['tags'] != []:
                rows.append({'code':code, 'labels':problem['tags']})
                index.append(submission)

data = DataFrame(rows, index=index)

# Split data into a training and testing set

In [86]:
vectorizer = CountVectorizer()
labels_vect = CountVectorizer()

BoW = vectorizer.fit_transform(dictionary)

features = vectorizer.transform(data['code'].values)
targets = data['labels'].values

# create traing and testing data
seed = 42
data_train, data_test = train_test_split(data,test_size=0.20, random_state=seed)
data_train.to_csv('out.csv')
x_train = vectorizer.transform(data_train['code'].values)
y_train = MultiLabelBinarizer().fit_transform(data_train['labels'].values)
x_test = vectorizer.transform(data_test['code'].values)
y_test = MultiLabelBinarizer().fit_transform(data_test['labels'].values)

# Create Machine Learning Algorithms

In [119]:
svm_classifier = OneVsRestClassifier(svm.SVC(kernel='linear', class_weight='balanced'))
dt_classifier = tree.DecisionTreeClassifier(class_weight='balanced')
rf_classifier = RandomForestClassifier(n_estimators=10,class_weight='balanced')
nn_classifier = MLPClassifier(hidden_layer_sizes=(100,70),max_iter=500)

In [120]:
svm_classifier.fit(x_train,y_train)
dt_classifier.fit(x_train,y_train)
rf_classifier.fit(x_train,y_train)
nn_classifier.fit(x_train,y_train)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100, 70), learning_rate='constant',
       learning_rate_init=0.001, max_iter=500, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [122]:
score = svm_classifier.score(x_test, y_test)
print('SVM score: ' + str(score))
score = dt_classifier.score(x_test, y_test)
print('DT score: ' + str(score))
score = rf_classifier.score(x_test, y_test)
print('RF score: ' + str(score))
score = nn_classifier.score(x_test, y_test)
print('NN score: ' + str(score))

SVM score: 0.5267489711934157
DT score: 0.7791495198902606
RF score: 0.6872427983539094
NN score: 0.6995884773662552


In [123]:
svm_y_pred = svm_classifier.predict(x_test)
for i in range(y_test.shape[1]):
    display(metrics.confusion_matrix(y_test[:,i], svm_y_pred[:,i]))

array([[717,   3],
       [  2,   7]])

array([[592,  57],
       [ 15,  65]])

array([[669,  27],
       [ 10,  23]])

array([[721,   3],
       [  1,   4]])

array([[713,   5],
       [  5,   6]])

...

array([[457,  60],
       [ 18, 194]])

array([[715,   7],
       [  1,   6]])

array([[669,  30],
       [  7,  23]])

array([[635,  40],
       [ 13,  41]])

array([[711,   6],
       [  5,   7]])

In [104]:
dt_y_pred = dt_classifier.predict(x_test)
for i in range(y_test.shape[1]):
    display(metrics.confusion_matrix(y_test[:,i], dt_y_pred[:,i]))

array([[720,   0],
       [  3,   6]])

array([[629,  20],
       [ 11,  69]])

array([[690,   6],
       [ 10,  23]])

array([[718,   6],
       [  1,   4]])

array([[717,   1],
       [  2,   9]])

...

array([[486,  31],
       [ 14, 198]])

array([[718,   4],
       [  1,   6]])

array([[690,   9],
       [  9,  21]])

array([[655,  20],
       [ 35,  19]])

array([[712,   5],
       [  3,   9]])

In [118]:
print(np.array(y_test).toarray())
CCA(n_components=2).fit(x_test, y_test).transform(x_test)

AttributeError: 'numpy.ndarray' object has no attribute 'toarray'